In [1]:
#hide
!pip install matplotlib
!pip install pandas
!pip install requests
!pip install datetime

!pip install scipy
!pip install plotly


You should consider upgrading via the 'python -m pip install --upgrade pip' command.

You should consider upgrading via the 'python -m pip install --upgrade pip' command.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.

You should consider upgrading via the 'python -m pip install --upgrade pip' command.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
#hide
import os
QUERY_TIME="&from=now-6d&to=now"
K8sAddress='http://k8s00:30670/render?target='
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
directory="/" + os.path.join("data","sitespeedio")
targets= []
for site in os.listdir(directory):
    if site!= "Report":
        targets.append( site.replace(".","_"))
targets



['remaxdev-bonfim_devscope_net',
 'remaxdev-francos_devscope_net',
 'www_century21_pt',
 'www_remax_pt']

In [3]:
#hide
import requests
from datetime import datetime
import altair as alt
def getData(target, time):
    #print('http://k8s00:30670/render?target='+target+time +'&format=json')
    r = requests.get(K8sAddress+target+time +'&format=json')
    result=[]
    a= r.json()
    #print(a)
    a= a[0]
    for line in a.get("datapoints"):
        if str(line [0]).find("None") == -1 and len(result) > 6:
            result.append(line)
    return result

def getDataFrame(targets, graphite):
    index=[]
    timestamps= getData(graphite[0] + targets[0] +graphite[1] , QUERY_TIME)
    for timestamp in timestamps:
        index.append( datetime.fromtimestamp(timestamp[1]))
    df = pd.DataFrame(columns=targets, index=index )
    for target in targets:
        list = getData(graphite[0] + target +graphite[1] , QUERY_TIME)
        for num in range(len(list)):
            df[target][index[num]]= list[num][0]
    return df





In [4]:
from plotly.offline import plot as py
from plotly.graph_objects import Scatter
from plotly.graph_objects import Line
from plotly.graph_objects import Layout
from plotly.graph_objects import Data
from plotly.graph_objects import Figure
def interactiveChart(dfs , labels):
    colors=['#FF6633', '#FFB399', '#FF33FF', '#FFFF99', '#00B3E6',
		  '#E6B333', '#3366E6', '#999966', '#99FF99', '#B34D4D',
		  '#80B300', '#809900', '#E6B3B3', '#6680B3', '#66991A',
		  '#FF99E6', '#CCFF1A', '#FF1A66', '#E6331A', '#33FFCC',
		  '#66994D', '#B366CC', '#4D8000', '#B33300', '#CC80CC',
		  '#66664D', '#991AFF', '#E666FF', '#4DB3FF', '#1AB399',
		  '#E666B3', '#33991A', '#CC9999', '#B3B31A', '#00E680',
		  '#4D8066', '#809980', '#E6FF80', '#1AFF33', '#999933',
		  '#FF3380', '#CCCC00', '#66E64D', '#4D80CC', '#9900B3',
		  '#E64D66', '#4DB380', '#FF4D4D', '#99E6E6', '#6666FF']
    traces=[]
    buttons=[]
    num=0
    for df in dfs:
        visible=False
        if num==0 :
            visible=True
        for target in list(df.columns):
            traces.append(Scatter(
            x=list(df.index), y=df[target],
            line=Line(
                color=colors[targets.index(target)],
                width=3
            ),
            name=target,
            visible=visible,
            ))
        visibility=[]
        #print(len(df.columns))
        for i in range(num*len(df.columns)):
            visibility.append(False)
        for i in range(len(list(df.columns))):
            visibility.append(True)
        while len(visibility) < len(dfs)*len(list(df.columns)):
            visibility.append(False)
        print(visibility)
        buttons.append(dict(
                        args=['visible', visibility],
                        label=labels[num],
                        method='restyle'
                    ))
        num+=1


    data = Data(traces)
    layout = Layout(
        title='Sitespeedio report',
        updatemenus=list([
            dict(
                x=-0.05,
                y=1,
                yanchor='top',
                buttons=buttons,
            )
        ]),
    )
    fig = Figure(data=data, layout=layout)
    py(fig)



In [5]:

#hide_input
import pandas as pd
import numpy as np
import scipy as sp
from plotly.offline import plot as py
from plotly import figure_factory as ff


firstPaintdf = getDataFrame(targets , ["sitespeed_io.default.summary." ,".chrome.native.browsertime.firstPaint.mean"])

firstPaintdf

,remaxdev-bonfim_devscope_net,remaxdev-francos_devscope_net,www_century21_pt,www_remax_pt


In [6]:
#hide_input
cpuLongTasks = getDataFrame(targets , ["sitespeed_io.default.summary.",".chrome.native.browsertime.cpu.longTasks.totalBlockingTime.mean"])

cpuLongTasks

,remaxdev-bonfim_devscope_net,remaxdev-francos_devscope_net,www_century21_pt,www_remax_pt


In [7]:
#hide_input
coach = getDataFrame(targets , ["sitespeed_io.default.summary.",".chrome.native.coach.score.mean"])

coach


,remaxdev-bonfim_devscope_net,remaxdev-francos_devscope_net,www_century21_pt,www_remax_pt


In [8]:
#hide_input
dfs =[firstPaintdf, cpuLongTasks , coach]
interactiveChart(dfs , ["Paint Time" , "CPU long tasks" , "Coach score"])


C:\Python38\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


[True, True, True, True, False, False, False, False, False, False, False, False]
[False, False, False, False, True, True, True, True, False, False, False, False]
[False, False, False, False, False, False, False, False, True, True, True, True]
C:\Python38\lib\site-packages\plotly\graph_objs\_deprecations.py:31: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




In [9]:
#hide_input
from IPython.display import IFrame
IFrame('./temp-plot.html', width=1000, height=500)